In [13]:
import re
import numpy as np
import pandas as pd

# csv 경로
---

In [14]:
# 파일 경로
file_path = './data/'
df = pd.read_csv(f'{file_path}survey_results_public.csv')

# 사용할 컬럼
cols = ['LearnCode_count'
        ,'Lang_Diversity', 'AI_Tool_Count', 'WorkExp', 'YearsCode' 
        ,'DevRole_Count'
        ,'SOHow_count' 
        ,'SOComm_encoded', 'NEWSOSites_count' 
        ,'Age_encoded'
        ,'Challenges_count', 'AIForecastScore', 'AIThreat_num'
        ,'is_churned']
print(len(cols))

14


# def
---

In [15]:
# 복수 응답 개수 계산
# ";" 또는 "," 개수 + 1로 계산함 / 빈 문자열이거나 NaN(None, float('nan'))은 0으로 간주
def countMultipleResponses(value):
    if pd.isna(value) or str(value).strip() == "":
        return 0
    return len(re.findall(r"[;,]", str(value))) + 1

# 타겟 변수
- SOVisitFreq(방문 빈도)가 "매달 1번 이하", "Stack Overflow에 방문하지 않음" 을 이탈로 간주해서 타겟으로 설정
---

In [16]:
# SOVisitFreq 결측치 있는 행 제거
df = df.dropna(subset=['SOVisitFreq'])
df['is_churned'] = df['SOVisitFreq'].apply(lambda x: 1 if x in ['Less than once per month or monthly', 'I don’t visit Stack Overflow'] else 0)
# 이탈 1

# 추가

In [17]:
# 언어 학습 방법
df['LearnCode_count'] = df['LearnCode'].apply(countMultipleResponses)

# 민영
---
'Lang_Diversity_Scaled', 'AI_Tool_Count_Scaled', 'Pro_Years_Scaled', 'Years_Scaled', 'DevRole_Scaled'

In [18]:
# 1. 사용 언어 다양성: 사용한 프로그래밍 언어 수 카운트
# ───────────────────────────────────────────────
df['Lang_Diversity'] = df['LanguageHaveWorkedWith'].apply(countMultipleResponses)

# 2. AI 도구 사용 수: 현재 사용하는 AI 도구 개수 카운트
# ───────────────────────────────────────────────
df['AI_Tool_Count'] = df['AIToolCurrently Using'].apply(countMultipleResponses)

# 3. 실무 경력
# ───────────────────────────────────────────────
# 문자열 → 숫자 변환, 결측치 0 으로 처리
df['WorkExp'] = pd.to_numeric(df['WorkExp'], errors='coerce')   # float 형으로 변경
df['WorkExp'] = df['WorkExp'].fillna(0)

# 4. 코딩 경험 연차: YearsCode 처리
# ───────────────────────────────────────────────
# 코딩경험 float 형으로 변경, 결측치는 0 로 간주
df['YearsCode'] = df['YearsCode'].replace({
    'Less than 1 year': 0.5,
    'More than 50 years': 51
}).astype(float).fillna(0)

# 5. 직무 역할 수: DevType에서 역할 개수 카운트
# ───────────────────────────────────────────────
df['DevRole_Count'] = df['DevType'].apply(countMultipleResponses)

In [19]:
df['YearsCode'].value_counts().sort_index()

YearsCode
0.0     1794
0.5      448
1.0      615
2.0     1530
3.0     2368
4.0     3128
5.0     3550
6.0     3345
7.0     3218
8.0     3334
9.0     2175
10.0    4416
11.0    1588
12.0    2487
13.0    1434
14.0    1789
15.0    2740
16.0    1363
17.0    1013
18.0    1200
19.0     544
20.0    2568
21.0     511
22.0     829
23.0     617
24.0     849
25.0    1634
26.0     618
27.0     494
28.0     503
29.0     271
30.0    1419
31.0     142
32.0     327
33.0     218
34.0     292
35.0     724
36.0     217
37.0     207
38.0     282
39.0     143
40.0     982
41.0     143
42.0     284
43.0     172
44.0     194
45.0     250
46.0      93
47.0      59
48.0      64
49.0      34
50.0      75
51.0     242
Name: count, dtype: int64

# 길진
---
'SOHow_count', 'SOComm_encoded', 'NEWSOSites_count', 'is_churned'

In [20]:
# 1. 커뮤니티 방문 사유 수: SOHow 응답 개수 카운트
# ───────────────────────────────────────────────
df['SOHow_count'] = df['SOHow'].apply(countMultipleResponses)

# 2. 커뮤니티 인식 수준: SOComm 응답 인코딩 (0~5)
# ───────────────────────────────────────────────
so_comm_order = {
    'No, not at all': 0,
    'No, not really': 1,
    'Not sure': 2,
    'Neutral': 3,
    'Yes, somewhat': 4,
    'Yes, definitely': 5
}

df['SOComm_encoded'] = df['SOComm'].fillna('Neutral').map(so_comm_order)

# 3. Stack Overflow 사이트 방문 수: NEWSOSites 응답 개수 카운트
# ───────────────────────────────────────────────
df['NEWSOSites_count'] = df['NEWSOSites'].apply(countMultipleResponses)

# 의중
---
<br>
'Challenges_count', 'AIThreat_num'

In [21]:
# 1. 결측치 처리: 지정한 컬럼의 null/None 값을 np.nan으로 통일
# ───────────────────────────────────────────────
for col in [
    'AIToolCurrently Using',
    'AINextMuch more integrated',
    'AINextMore integrated',
    'AIChallenges'
]:
    df[col] = df[col].where(pd.notnull(df[col]), np.nan)

# 2. AI 도구 사용 장애 요인 수: 다중 응답 개수 카운트
# ───────────────────────────────────────────────
df['Challenges_count']= df['AIChallenges'].apply(countMultipleResponses)


# 3. 향후 1년 내에 AI 도구가 얼마나 통합될거 같은지에 대한 점수?
# ─────────────────────────────────────────────── 
df['AINextMuch_more_count'] = df['AINextMuch more integrated'].apply(countMultipleResponses)
df['AINextMore_count'] = df['AINextMore integrated'].apply(countMultipleResponses)
df['AINextLess_count'] = df['AINextLess integrated'].apply(countMultipleResponses)
df['AINextMuch_less_count'] = df['AINextMuch less integrated'].apply(countMultipleResponses)

df['AIForecastScore'] = (
    df['AINextMuch_more_count'] * 2 
    + df['AINextMore_count']
    - df['AINextLess_count']
    - df['AINextMuch_less_count'] * 2
)

# 4. AI의 위협 인식: 결측치 처리 후 숫자 라벨 인코딩
# ───────────────────────────────────────────────
# 결측치 "I’m not sure"로 처리 후 숫자 인코딩
df['AIThreat'] = df['AIThreat'].fillna("I'm not sure")
df['AIThreat_num'] = df['AIThreat'].map({
    'Yes': 1,
    'No': 0,
    "I'm not sure": 0.5
})

# 주서

---
'Age_encoded'

In [ ]:
# 나이 구간별 라벨 인코딩
# ───────────────────────────────────────────────
age_order = {
    'Under 18 years old': 0,
    '18-24 years old': 1,
    '25-34 years old': 2,
    '35-44 years old': 3,
    '45-54 years old': 4,
    '55-64 years old': 5,
    '65 years or older': 6,
    'Prefer not to say': np.nan
}

df['Age_encoded'] = df['Age'].map(age_order)
df = df.dropna(subset=['Age_encoded'])
# df['Age_encoded'] = df['Age_encoded'].astype(int)

# 결측치 처리 (임시)
---

In [23]:
df['SOComm_encoded'] = df['SOComm_encoded'].fillna(0)

# 전처리한 파일 저장
---

In [26]:
df_fin = df[cols].dropna()
print(df_fin.info())
df_fin.to_csv(f'{file_path}survey_results_cleaned_scaler.csv', index=False, encoding='utf-8-sig')

<class 'pandas.core.frame.DataFrame'>
Index: 59265 entries, 1 to 65436
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LearnCode_count   59265 non-null  int64  
 1   Lang_Diversity    59265 non-null  int64  
 2   AI_Tool_Count     59265 non-null  int64  
 3   WorkExp           59265 non-null  float64
 4   YearsCode         59265 non-null  float64
 5   DevRole_Count     59265 non-null  int64  
 6   SOHow_count       59265 non-null  int64  
 7   SOComm_encoded    59265 non-null  int64  
 8   NEWSOSites_count  59265 non-null  int64  
 9   Age_encoded       59265 non-null  float64
 10  Challenges_count  59265 non-null  int64  
 11  AIForecastScore   59265 non-null  int64  
 12  AIThreat_num      59265 non-null  float64
 13  is_churned        59265 non-null  int64  
dtypes: float64(4), int64(10)
memory usage: 6.8 MB
None
